In [1]:
from model.types import MarketDemandSupply
from model.run import load_config, run, post_processing

from model.mechanism.amm_k import amm_k_mechanism
from model.mechanism.treasury import liq_backing_mechanism,treasury_stables_mechanism,floating_supply_mechanism,liq_ohm_mechanism
from model.policy.rbs_price import lower_target_policy,upper_target_policy
initialstate = { # variable values borrowed from liquidity-olympus/simulation.ipynb
            'supply':30000000,
            "liq_stables": 21000000,
            "reserves_stables":200000000,
            "reserves_volatile":17000000,
            "price":9,

            "market_demand_supply": MarketDemandSupply(total_supply = 50,
                                                   total_demand = 50),
                                                   
            # treasury policy
            'target_liq_ratio_reached':False,
            'reserves_in':0,
            'supply':25000000,

            # RBS related
            'ma_target':9,
            # market transaction variables
            'ask_change_ohm':0,
            'bid_change_ohm':0,
            "net_flow": None
            }
# initialize parameters that are decided by other parameters
initialstate['amm_k'] = amm_k_mechanism(initialstate)
initialstate['price_history']=[initialstate['price']]

initialstate['lb_target'] = initialstate['ma_target']
initialstate['price_target']=initialstate['ma_target']

initialstate['liq_ohm'] = liq_ohm_mechanism(initialstate['liq_stables'],initialstate['price'])
initialstate['floating_supply'] = floating_supply_mechanism(initialstate['supply'],initialstate['liq_ohm'])

params = {"demand_factor": [10],
         "supply_factor": [10],
         "initial_reserves_volatile":[17000000], # setting it as constant since in this simulation we don't use it at all.
         "reward_rate":[0.000198], # flat reward rate; (1 + r) ^ 365 ~ 7.5%
         "max_liq_ratio":[0.14375],  # LiquidityUSD : reservesUSD ratio --> 1:1 = 0.5
         "target_ma":[30], # number of days
         "lower_wall":[0.15],
         "upper_wall":[0.15],
         "lower_cushion":[.075],
         "upper_cushion":[.075],
         }

initialstate['treasury_stables'] = treasury_stables_mechanism(initialstate['liq_stables'],initialstate['reserves_stables'])
initialstate['liq_backing'] = liq_backing_mechanism(initialstate['treasury_stables'],params['initial_reserves_volatile'][0])
initialstate['target_walls'] = [lower_target_policy(initialstate['price_target'], params['lower_wall'][0]),
                                upper_target_policy(initialstate['price_target'], params['upper_wall'][0])]
initialstate['target_cushions'] = [lower_target_policy(initialstate['price_target'], params['lower_cushion'][0]),
                                    upper_target_policy(initialstate['price_target'], params['upper_cushion'][0])]
initialstate['bid_counter'],initialstate['ask_counter'] = [],[]

nrun = 5
nstep = 10
exp = load_config(nrun, params, initialstate, nstep)
raw = run(exp)
df = post_processing(raw)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 10, 10, 5, 24)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (10, 10, 5, 24)
Execution Method: local_simulations
Execution Mode: parallelized
Total execution time: 0.41s


In [8]:
raw.columns

Index(['supply', 'liq_stables', 'reserves_stables', 'reserves_volatile',
       'price', 'market_demand_supply', 'target_liq_ratio_reached',
       'reserves_in', 'ma_target', 'ask_change_ohm', 'bid_change_ohm',
       'net_flow', 'amm_k', 'price_history', 'lb_target', 'price_target',
       'liq_ohm', 'floating_supply', 'treasury_stables', 'liq_backing',
       'target_walls', 'target_cushions', 'bid_counter', 'ask_counter',
       'simulation', 'subset', 'run', 'substep', 'timestep'],
      dtype='object')

0      221000000
1      221000000
2      221000000
3      221000000
4      221000000
         ...    
600    221000000
601    221000000
602    221000000
603    221000000
604    221000000
Name: treasury_stables, Length: 605, dtype: int64

In [17]:
left = raw[raw["substep"] == 1][['liq_stables', 'reserves_stables']].sum(axis=1)
right = raw[raw["substep"] == 1]['treasury_stables']

assert (left == right).all()

In [6]:
sample

,supply,liq_stables,reserves_stables,reserves_volatile,price,market_demand_supply,target_liq_ratio_reached,reserves_in,ma_target,ask_change_ohm,...,liq_backing,target_walls,target_cushions,bid_counter,ask_counter,simulation,subset,run,substep,timestep
0,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[7.6499999999999995, 10.35]","[8.325000000000001, 9.674999999999999]",[],[],0,0,1,0,0
1,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[7.6499999999999995, 10.35]","[8.325000000000001, 9.674999999999999]",[],[],0,0,1,1,1
2,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[7.6499999999999995, 10.35]","[8.325000000000001, 9.674999999999999]",[],[],0,0,1,2,1
3,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[7.6499999999999995, 10.35]","[8.325000000000001, 9.674999999999999]",[],[],0,0,1,3,1
4,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[7.6499999999999995, 10.35]","[8.325000000000001, 9.674999999999999]",[0],[],0,0,1,4,1
5,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[7.6499999999999995, 10.35]","[8.325000000000001, 9.674999999999999]",[0],[0],0,0,1,5,1
6,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[7.6499999999999995, 10.35]","[8.325000000000001, 9.674999999999999]",[0],[0],0,0,1,6,1
7,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[7.6499999999999995, 10.35]","[8.325000000000001, 9.674999999999999]",[0],[0],0,0,1,7,1
8,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[8.924999999999999, 12.075]","[8.325000000000001, 9.674999999999999]",[0],[0],0,0,1,8,1
9,2.500000e+07,21000000,200000000,17000000,9,"MarketDemandSupply(total_supply=50, total_dema...",False,0.0,9.0,0,...,238000000,"[8.924999999999999, 12.075]","[9.7125, 11.2875]",[0],[0],0,0,1,9,1
